In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
import numpy as np

print(f"tensorflow {tf.__version__}")
print(f"Numpy {np.__version__}")

tensorflow 2.1.0
Numpy 1.19.2


In [ ]:
BATCH_SIZE = 64,
ORIGINAL_DIM = 28 * 28

LATENT_DIM = 2 # tHE cOMPRESSED dIMENSION
INTERMEDIATE_DIM = 256 # The units For Decoder
EPOCHS = 5
epsilon_std = 1.0 # For generating The Gaussian 


